## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from tensorflow.keras.callbacks import EarlyStopping

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 120
application_types_to_replace = list(application_counts[application_counts <= cutoff_value].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classification_counts = classification_counts[classification_counts > 1]
print(filtered_classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts <= cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_converted = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], drop_first=True)

# Create the target array y by extracting the 'IS_SUCCESSFUL' column
application_df_converted


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_converted['IS_SUCCESSFUL'].values
X = application_df_converted.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

# Define the number of neurons for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of neurons for the second hidden layer
hidden_nodes_layer2 = 32

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                2816      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,929
Trainable params: 4,929
Non-trainable params: 0
_________________________________________________________________


In [13]:
# # Compile the model
# nn.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
# nn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# nn.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])
# nn.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
# nn.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
# nn.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])
# nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# # Train the model
# num_epochs = 100
# batch_size = 32

# history = nn.fit(
#     X_train_scaled,
#     y_train,
#     epochs=num_epochs,
#     batch_size=batch_size,
#     verbose=1,
#     validation_split=0.2
# )


In [15]:
# Define a list of optimizers
optimizers = [SGD(), Adam(), RMSprop(), Adagrad(), Adadelta(), Adamax(), Nadam()]

In [16]:
# Create DataFrame to store the results
results_df = pd.DataFrame(columns=['optimizer', 'train_loss', 'train_accuracy', 'val_loss', 'val_accuracy'])

>The following code adds an EarlyStopping callback with a patience of 10 epochs, meaning that the training will stop if the validation loss doesn't improve for 10 consecutive epochs. This can significantly speed up the training process without compromising the integrity of the model.

In [17]:
num_epochs = 100
batch_size = 32

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Loop through the optimizers
for optimizer in optimizers:
    # Create and compile the model with the current optimizer
    nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # Train the model with early stopping
    history = nn.fit(
        X_train_scaled,
        y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        verbose=0,
        validation_split=0.2,
        callbacks=[early_stopping]
    )
    
    # Save the results to the DataFrame
    results_df = results_df.append({
        'optimizer': str(optimizer),
        'train_loss': history.history['loss'][-1],
        'train_accuracy': history.history['accuracy'][-1],
        'val_loss': history.history['val_loss'][-1],
        'val_accuracy': history.history['val_accuracy'][-1]
    }, ignore_index=True)


In [ ]:
# Save the DataFrame to a CSV file
results_df.to_csv('optimizer_results.csv',
                  index=False,
                  header=True,
                  float_format='%.4f')


In [ ]:
# Print the results
print(results_df)

In [ ]:
# Play a sound to let you know its done 
import winsound
duration = 2000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [ ]:
# import openpyxl
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl.styles import Font, Alignment, PatternFill, Border, Side

# # Create a new Excel workbook and add a worksheet
# wb = openpyxl.Workbook()
# ws = wb.active
# ws.title = "Optimizer Results"

# # Write the DataFrame to the worksheet
# for r in dataframe_to_rows(results_df, index=False, header=True):
#     ws.append(r)

# # Apply formatting to the header row
# for cell in ws["1:1"]:
#     cell.font = Font(bold=True)
#     cell.alignment = Alignment(horizontal="center", vertical="center")
#     cell.fill = PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
#     cell.border = Border(left=Side(style='thin'), 
#                          right=Side(style='thin'), 
#                          top=Side(style='thin'), 
#                          bottom=Side(style='thin'))

# # Apply formatting to the data cells
# for row in ws.iter_rows(min_row=2):
#     for cell in row:
#         cell.number_format = '0.0000'
#         cell.alignment = Alignment(horizontal="right", vertical="center")
#         cell.border = Border(left=Side(style='thin'), 
#                              right=Side(style='thin'), 
#                              top=Side(style='thin'), 
#                              bottom=Side(style='thin'))

# # Adjust column widths
# for column_cells in ws.columns:
#     length = max(len(str(cell.value)) for cell in column_cells)
#     ws.column_dimensions[column_cells[0].column_letter].width = length + 2

# # Save the workbook to an Excel file
# wb.save('optimizer_results.xlsx')


In [ ]:
sorted_results = results_df.sort_values(by='train_accuracy', ascending=False)
sorted_results

In [ ]:
# Get the optimizer of the best-performing model
best_optimizer = sorted_results.iloc[0]['optimizer']
best_optimizer

In [ ]:
# Define the best model using the best optimizer
best_model = Sequential()
best_model.add(Dense(units=16, activation='relu', input_dim=X_train.shape[1]))
best_model.add(Dense(units=8, activation='relu'))
best_model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
optimizer_dict = {
    "Adam": Adam,
    "SGD": SGD,
    "RMSprop": RMSprop,
    "Adadelta": Adadelta,
    "Adagrad": Adagrad,
    "Adamax": Adamax,
    "Nadam": Nadam
}

In [ ]:
best_optimizer_string = results_df.loc[results_df['val_accuracy'].idxmax(), 'optimizer']

best_optimizer_string

In [ ]:
best_optimizer_instance = optimizer_dict[best_optimizer_string]()


In [ ]:
# Compile the best model
best_model.compile(optimizer=best_optimizer, loss='binary_crossentropy', metrics=['train_accuracy'])

In [ ]:
# Train the best model
best_model.fit(X_train_scaled, y_train, epochs=100, verbose=1)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export the best model to an HDF5 file
best_model.save('AlphabetSoupCharity_Optimization.h5')